# process the individual samples and extract p values

we cannot provide raw genotypes for the individual patients, so this script is only for reference

the way this script works is that it opens each individually called BCF file. It then extracts the useful information, identifies the heterozygous sites and then calculates a basic p-value for each patient.

the next script will merge the p-values per variant and produce the final script

# note that we cannot provide raw genotypes for the patients, so this script is here only as a reference and it cannot be run

In [1]:
import pandas as pd
import numpy as np
import hicstraw 
from multiprocessing import Pool
from functools import partial
import glob
import os
import plotly.express as px
import math
import matplotlib.pyplot as plt
from matplotlib import colors
from pandarallel import pandarallel
import cooler
import cooltools
import pybedtools as pbed
pandarallel.initialize()
import scipy
from statsmodels.stats import multitest
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.io as pio
import seaborn as sns
import gzip
import io
import pickle
from functools import reduce

os.makedirs("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/", exist_ok = True)
pbed.helpers.set_tempdir("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/")
bed_genome_file = "/mnt/iusers01/jw01/mdefscs4/hg38.genome"

plt.rcParams['svg.fonttype'] = 'none'

base_dir = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis"

INFO: Pandarallel will run on 28 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
metadata_ATAC = pd.read_csv(f"{base_dir}/metadata/cleaned_ATAC_metadata.csv", index_col=0)

In [3]:
def read_vcf(path):
    with gzip.open(path, 'rt') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

In [4]:
def extract_numbers(x):
    # split the string by ':'
    numbers = x.split(':')
    # extract the last two numbers
    num1, num2 = numbers[-1].split(',')
    return int(num1), int(num2)


def get_data_per_sample(ID_sample):
    dataframes = []

    # loop through the 22 chromosomes
    for chrom in range(1, 23):
        # open the dataframe for each chromosome
        test_file = f"/mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_combined_analysis/allele_specific_hic/ATAC_specific/called_bcf/{ID_sample}/{ID_sample}_chr{chrom}_filt_annotated.vcf.gz"
        df = read_vcf(test_file)
        # add the dataframe to the list
        dataframes.append(df)

    # concatenate the dataframes
    result = pd.concat(dataframes).reset_index(drop = True)

    result = result.drop(["FILTER", "QUAL", "INFO", "FORMAT"], axis = 1)
    result = result.rename(columns={result.columns[-1]: ID_sample})

    result[[ID_sample + '_A', ID_sample + '_B']] = pd.DataFrame(result[ID_sample].apply(extract_numbers).tolist(), columns=[ID_sample + '_A', ID_sample + '_B'])
    
    # filter rows here
    def get_good_row(row):
        if (row[ID_sample + '_A'] > 2) and (row[ID_sample + '_B'] > 2): # either of them needs to be above two
            tot = row[ID_sample + '_A']+row[ID_sample + '_B']
            if (row[ID_sample + '_A']/tot > 0.05) and (row[ID_sample + '_B']/tot > 0.05): # both of them need to be above 5%, this is in case actual counts are very high
                return True
        return False
    result = result[result.apply(get_good_row, axis = 1)]
    def retrieve_p_val(row):
        row[f"{ID_sample}_pval_greater"] = scipy.stats.binom_test(row[f"{ID_sample}_A"],row[f"{ID_sample}_B"] + row[f"{ID_sample}_A"],0.5,"greater")
        row[f"{ID_sample}_pval_less"] = scipy.stats.binom_test(row[f"{ID_sample}_A"],row[f"{ID_sample}_B"] + row[f"{ID_sample}_A"],0.5,"less")
        return row
    result = result.parallel_apply(retrieve_p_val, axis = 1)
    return result

all_results = {}
for sample in metadata_ATAC["id"].to_list():
    result = get_data_per_sample(sample)
    all_results[sample] = result

    print(sample, len(result))

NRHV151_CD4 21886
NRHV151_CD8 22192
NRHV321_CD4 4027
PsA4920_CD4 23011
PsA4945_CD8 11650
PsA4948_CD8 13267
PsA4953_CD8 45741
PsA4960_CD8 12760
NRHV321_CD8 16892
PsA4944_CD4 22369
PsA4947_CD8 13527
PsA4949_CD4 18293
PsA4954_CD8 27711
PsA4963_CD8 17832
NRHV238_CD4 26499
nrhv238_CD8_ 12359
PsA4950_CD4 17559
PsA4961_CD8 21091
PsA4964_CD8 10565
PsA5012_CD4 28813
NRHV326_CD8 21260
PsA4920_CD8 20468
PsA4952_CD8 23734
PsA4963_CD4 10514
PsA5008_CD4 24347
PsA5012_CD8 25585
PsA4941CD8 20015
PsA4944CD8 22952
PSA4946CD8 18533
PsA4952CD4 10183
PsA4954CD4 17933
PsA4962CD8 10243
PSA4964CD4 12899
PsA4965CD8 21027
PSA4969CD8 26397
PsA4969CD8_SF 20100
PSA5008CD8 26988
PsA5014CD4 24709
PsA5014CD8 27362
PsA5019CD8 20563
NRHV171CD4 14358
NRHV171CD8 14450
NRHV324CD4 26733
PsA4950CD8 7297
PsA4958CD4 21714
PsA4958CD8 15293
PsA5009CD4 19038
PsA5010CD4 28587
PsA5010CD8 33084
PsA5018CD4 28255
PsA5018CD8 25937
PsA5023CD8 27508
PsA5026CD4 21747
PsA5026CD8 25178
PsA4956CD4 24624
PsA4956CD8 25227
PsA4968CD8 24986
PsA

In [ ]:
pickle.dump(all_results, open(f"{base_dir}/ATAC_allelic_imbalance/all_p_val_called.pk", "wb"))

In [5]:
pickle.dump(all_results, open("/mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_combined_analysis/allele_specific_hic/ATAC_specific/all_p_val_called.pk", "wb"))